In [9]:
import cv2
import time
import os
import numpy as np
import cv2
import glob
import ntpath
import os
from PIL import Image
from numpy import asarray
import pickle


# Function to Extract Frames from Video

In [10]:
def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        #print(frame.shape)
        #print(frame[0])
        #print(np.amax(frame))
        #break
        # Write the results back to output location.
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), frame)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds for conversion." % (time_end-time_start))
            break
    return count

# Segmentation


In [11]:
def frameconversion():
    print ("\n Converting frames . . .\n")
    filelist=glob.glob('C:/Users/dar/Desktop/Midas - Multiple Objects/frames/*.jpg')
    for i in filelist:
        image = cv2.imread(i)
        #print(type(image))
        #print(image[0])

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        blurred = cv2.GaussianBlur(gray, (11, 11), cv2.BORDER_DEFAULT)
        ret,th1 = cv2.threshold(blurred,170,255,cv2.THRESH_BINARY)


        # gaussian_thresh= cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,201,0)

        #cv2.imshow("binary",th1)

        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        # original = image.copy()
        # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        # #canny = cv2.Canny(blurred, 120, 255, 1)
        # #kernel = np.ones((5,5),np.uint8)
        # #dilate = cv2.dilate(canny, kernel, iterations=1)
        filename=ntpath.basename(i)
        pre=filename.split(".")[0]
        fi='C:/Users/dar/Desktop/Midas - Multiple Objects/step2/'+pre+".jpg"
        cv2.imwrite(fi,th1)
    print ("Applied Gaussian Blur and Threshold . . .\n")

# Remove the FLIR Logo

In [12]:
def removeflir():
    filelist=glob.glob('C:/Users/dar/Desktop/Midas - Multiple Objects/step2/*.jpg')
    for i in filelist:
        image=cv2.imread(i)
        rectangle = np.array([[3,475],[160, 475], [165, 405], [7, 405]])
        color = [0, 0, 0] #black
        cv2.fillConvexPoly(image, rectangle, color)
        filename=ntpath.basename(i)
        pre=filename.split(".")[0]
        fi='C:/Users/dar/Desktop/Midas - Multiple Objects/step2/'+pre+".jpg"
        cv2.imwrite(fi,image)
    print ("Removed FLIR watermark . . .\n")

# Find ROIs

In [38]:
filelist=glob.glob('C:/Users/dar/Desktop/Midas - Multiple Objects/step2/*.jpg')
image_number = 0
for i in filelist:
    image = cv2.imread(i)
    original = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    canny = cv2.Canny(blurred, 120, 255, 1)
    kernel = np.ones((5,5),np.uint8)
    dilate = cv2.dilate(canny, kernel, iterations=1)

    # Find contours
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    obj=1
    for c in cnts:
        if(obj==1):
            x1,y1,w1,h1 = cv2.boundingRect(c)
            obj+=1
        else:
            x2,y2,w2,h2 = cv2.boundingRect(c)
    break

# Separate the Objects

In [32]:
def objectseparation():
    filelist=glob.glob('C:/Users/dar/Desktop/Midas - Multiple Objects/step2/*.jpg')
    image_number = 0
    for i in filelist:
        image = cv2.imread(i)
        original = image.copy()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        canny = cv2.Canny(blurred, 120, 255, 1)
        kernel = np.ones((5,5),np.uint8)
        dilate = cv2.dilate(canny, kernel, iterations=1)


        for j in range(2):
            if(j==0):
                x,y,w,h=x1,y1,w1,h1
                path="C:/Users/dar/Desktop/Midas - Multiple Objects/OBJ1/"
            else:
                x,y,w,h=x2,y2,w2,h2
                path="C:/Users/dar/Desktop/Midas - Multiple Objects/OBJ2/"
            #x,y,w,h = cv2.boundingRect(c)
            #print(x,y,w,h)
            cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
            #cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
            ROI = original[y-30:y+h+30, x-30:x+w+30]
            cv2.imwrite(path+"{}.jpg".format(image_number), ROI)
            image_number += 1
            #area = cv2.contourArea(c)
            #print(area)
    #     cv2.imshow('canny', canny)
    #     cv2.imshow('image', image)
    #     cv2.waitKey(0)

In [33]:
##Get Information about White Parts 
def datapreprocessing(objectlocation):
    filelist=glob.glob(objectlocation)
    result_array = np.array([])

    counter = 0
    for i in filelist:
        if counter == 20:
            counter = 0
            image = cv2.imread(i)
            occurrences = np.count_nonzero(image == 255)
            result_array = np.append(result_array, occurrences)
            #print(occurrences)
        counter += 1
    #print(type(result_array[0]))
    
    ## Normalise The DATA and save the Normalised Data to NPZ File

    startvalue=result_array[0]

    data=np.array([])

    counter = 0
    sample = []
    samples = []
    #extract normalised data
    for i in result_array:
        if counter == 10:
            samples.append(sample)
            sample = []
            counter = 0
        devvalue=float(startvalue-i)/float(startvalue)
    #     print(devvalue)
        normvalue=1-devvalue
        #print(normvalue)
        sample.append(normvalue)
        counter += 1

    samples_np = np.array(samples)
    samples_np.shape 
    print("Created .npz file")
    #     data=np.append(data,normvalue)
    return samples
    
   

In [34]:
def fixedholdprediction(test):
    data = test['samples']
    X=np.array(data)
    
    # Load the Saved Model back from file
    with open("C:/Users/dar/Desktop/Midas - Multiple Objects/saved models/Fixedhold_Modell.pkl", 'rb') as file:  
        Model = pickle.load(file)
    result=Model.predict(X)
    uniqueclasses,count_class=np.unique(result,return_counts=True)
    predictedindex=np.argmax(count_class)
    classresult=uniqueclasses[predictedindex]
    return classresult

In [35]:
def naturalholdprediction(test):
    data = test['samples']
    X=np.array(data)
    
    # Load the Saved Model back from file
    with open("C:/Users/dar/Desktop/Midas - Multiple Objects/saved models/Naturalhold_Modell.pkl", 'rb') as file:  
        Model = pickle.load(file)
    result=Model.predict(X)
    uniqueclasses,count_class=np.unique(result,return_counts=True)
    predictedindex=np.argmax(count_class)
    classresult=uniqueclasses[predictedindex]
    return classresult

In [36]:
def combinedprediction(test):
    data = test['samples']
    X=np.array(data)
    
    # Load the Saved Model back from file
    with open("C:/Users/dar/Desktop/Midas - Multiple Objects/saved models/Combined_Model.pkl", 'rb') as file:  
        Model = pickle.load(file)
    result=Model.predict(X)
    uniqueclasses,count_class=np.unique(result,return_counts=True)
    predictedindex=np.argmax(count_class)
    classresult=uniqueclasses[predictedindex]
    return classresult

In [39]:
if __name__=="__main__":

    input_loc = 'C:/Users/dar/Desktop/Midas - Multiple Objects/Video1.mp4'
    output_loc = 'C:/Users/dar/Desktop/Midas - Multiple Objects/frames'
    video_to_frames(input_loc, output_loc)
    frameconversion()
    removeflir()
    objectseparation()
    samplesobj1=datapreprocessing('C:/Users/dar/Desktop/Midas - Multiple Objects/OBJ1/*.jpg')
    np.savez_compressed('npzfileobj1', samples=samplesobj1)   #name of .npz file
    samplesobj2=datapreprocessing('C:/Users/dar/Desktop/Midas - Multiple Objects/OBJ2/*.jpg')
    np.savez_compressed('npzfileobj2', samples=samplesobj2)   #name of .npz file
    
    #Load the File for Object 1 
    test = np.load('npzfileobj1.npz')
    print("\nFor Object 1: \n")
    result1=fixedholdprediction(test)
    result2=naturalholdprediction(test)
    result3=combinedprediction(test)
    print("Fixed Hold Model Result: The material belongs to Class : ", result1)
    print("Natural Hold Model Result: The material belongs to Class : ", result2)
    print("Combined Model Result: The material belongs to Class : ", result3)
    
    #Load the File for Object 2
    test1 = np.load('npzfileobj2.npz')
    print("\nFor Object 2: \n")
    result11=fixedholdprediction(test1)
    result22=naturalholdprediction(test1)
    result33=combinedprediction(test1)
    print("Fixed Hold Model Result: The material belongs to Class : ", result11)
    print("Natural Hold Model Result: The material belongs to Class : ", result22)
    print("Combined Model Result: The material belongs to Class : ", result33)
    

Number of frames:  876
Converting video..

Done extracting frames.
876 frames extracted
It took 7 seconds for conversion.

 Converting frames . . .

Applied Gaussian Blur and Threshold . . .

Removed FLIR watermark . . .

Created .npz file
Created .npz file

For Object 1: 

Fixed Hold Model Result: The material belongs to Class :  1
Natural Hold Model Result: The material belongs to Class :  1
Combined Model Result: The material belongs to Class :  1

For Object 2: 

Fixed Hold Model Result: The material belongs to Class :  1
Natural Hold Model Result: The material belongs to Class :  1
Combined Model Result: The material belongs to Class :  1
